### Import files

In [25]:
import numpy as np
import pandas as pd
import os
import statistics
import heapq

path = "C:\\Users\pault\OneDrive\Documenten\GitHub\input" # pauls path
#add your path here
os.chdir(path)

train = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
print("train size:", train.size)

test = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')
print("test size:", test.size)

train size: 11860280
test size: 4853513


### Small test space

In [24]:
import statistics
print(train.size)

train.get_value(1, "price")
a = statistics.mean([1, 4, 5, 6, 4, 3, 2])
print(a)

train.head(100)

11860280
3.5714285714285716


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity
5,5,Bundled items requested for Ruie,3,Women/Other/Other,NaN,59.0,0,"Banana republic bottoms, Candies skirt with ma..."
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,0,Girls Size small Plus green. Three shorts total.
9,9,Porcelain clown doll checker pants VTG,3,Vintage & Collectibles/Collectibles/Doll,NaN,8.0,0,I realized his pants are on backwards after th...


### Functions

In [59]:
sample = train.loc[0:10]

# sorts a column and returns a dictionary with a list of indexes that have the same text/number in that column. 
def sort_index_by_column(data, column):
    dct = {'nan': []}
    for index, row in data.iterrows():
        if row[column] == row[column]:
            if row[column] in dct: 
                dct[row[column]].append(index)
            else: 
                dct[row[column]] = [index]
        else:
            dct["nan"].append(index)
    return dct

#deletes all the keys that have a list with only one value 
def delete_einzelgangers(dictionary):
    new_dict = {}
    for key in dictionary:
        if len(dictionary[key]) != 1:
               new_dict[key] = dictionary[key]
    return new_dict

# for every list, replace all the indexes with prices
def replace_index_with_price(dicti, data):
    new_dict = {}
    for key in dicti: 
        lst = []
        for ind in dicti[key]: 
            lst.append(data.get_value(ind, "price"))
        new_dict[key] = lst
    return new_dict

# calculates various things for every key
def mean_var_amount_per_dict_key(dicti):
    mean_dict = {}
    for key in dicti: 
        mean = statistics.mean(dicti[key])
        variance = statistics.variance(dicti[key])
        amount = len(dicti[key])
        mean_dict[key] = [mean, variance, int(amount)]
    return mean_dict

# gets the amount of times a word is being used in the item description
def get_common_words(data):
    dct = {}
    for index, row in data.iterrows():
        for word in row["item_description"].split(" "):
            if word in dct: 
                dct[word] += 1
            else: 
                dct[word] = 1
    highest_keys = sorted(dct, key=dct.get, reverse=True)
    new_dct = {}
    for key in highest_keys:
        new_dct[key] = dct[key]
    return new_dct

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

# every item (except a few NaN's) have three categories. Here we split those in three different columns.
def split_categories(data):
    data["first_cat"] = ""
    data["second_cat"] = ""
    data["third_cat"] = ""
    for index, row in data.iterrows():
        if row["category_name"] == row["category_name"]:
            data.set_value(index, "first_cat", row["category_name"].split("/")[0])
            data.set_value(index, "second_cat", row["category_name"].split("/")[1])
            data.set_value(index, "third_cat", row["category_name"].split("/")[2])
        print(row)
    return data

### Print stuff

In [57]:
sort_brand_name_ind = delete_einzelgangers(sort_index_by_column(sample, "brand_name"))
# for key in sort_brand_name_ind: 
#     print(key + ":", sort_brand_name_ind[key])  

sort_brand_name_price = replace_index_with_price(sort_brand_name_ind, sample)
# for key in sort_brand_name_price: 
#     print(key + ":", sort_brand_name_price[key])
    
sort_brand_name_mva = mean_var_amount_per_dict_key(sort_brand_name_price)
# for key in sort_brand_name_mva:
#     print(key + " - mean:", int(sort_brand_name_mva[key][0]), "variance:", 
#     int(sort_brand_name_mva[key][1]), "amount:", sort_brand_name_mva[key][2])
        
words = get_common_words(sample)
#for word in words: 
#    print(word + ":", words[word])

# for index, row, in sample.iterrows():
#     descr = row["item_description"]
#     if hasNumbers(descr) == True: 
#         print(row["name"], descr, "\n")



In [65]:
sample.head(5)
#train.head(5)
        
b = split_categories(sample)
b.head(5)



train_id                                               0
name                 MLB Cincinnati Reds T Shirt Size XL
item_condition_id                                      3
category_name                          Men/Tops/T-shirts
brand_name                                           NaN
price                                                 10
shipping                                               1
item_description                      No description yet
first_cat                                               
second_cat                                              
third_cat                                               
Name: 0, dtype: object
train_id                                                             1
name                                  Razer BlackWidow Chroma Keyboard
item_condition_id                                                    3
category_name        Electronics/Computers & Tablets/Components & P...
brand_name                                                       R

C:\Code Languages\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Code Languages\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Code Languages\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,first_cat,second_cat,third_cat
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces
